# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Liu  ->  J. Liu  |  ['J. Liu']
C. Vázquez  ->  C. C. Vázquez  |  ['C. Vázquez']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
J. Li  ->  J. Li  |  ['J. Li']
I. J. M. Crossfield  ->  I. J. M. Crossfield  |  ['I. J. M. Crossfield']
J. Eberhardt  ->  J. Eberhardt  |  ['J. Eberhardt']
T. Henning  ->  T. Henning  |  ['T. Henning']
Arxiv has 71 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2412.08057
extracting tarball to tmp_2412.08057... done.
Retrieving document from  https://arxiv.org/e-print/2412.08333


extracting tarball to tmp_2412.08333... done.
Retrieving document from  https://arxiv.org/e-print/2412.08390
extracting tarball to tmp_2412.08390... done.
Retrieving document from  https://arxiv.org/e-print/2412.08465


extracting tarball to tmp_2412.08465... done.
  2: tmp_2412.08465/natnotes.tex, 332 lines
  3: tmp_2412.08465/Mstars.tex, 732 lines
  5: tmp_2412.08465/natbib.tex, 96 lines
  6: tmp_2412.08465/aassymbols.tex, 579 lines


/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found 4 candidates with documentclass definition.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Assuming tmp_2412.08465/Mstars.tex as main document.
  exec(code_obj, self.user_global_ns, self.user_ns)


...

Found 100 bibliographic references in tmp_2412.08465/Mstars.bbl.
Error retrieving bib data for 2024AJ....167..174C: 'author'
Error retrieving bib data for 2024AJ....167..174C: 'author'
Retrieving document from  https://arxiv.org/e-print/2412.08527


extracting tarball to tmp_2412.08527...

 done.


/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2412.08527/ref.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'toi4168' from 'tmp_2412.08527/toi4168.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


Found 181 bibliographic references in tmp_2412.08527/ref.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.08465-b31b1b.svg)](https://arxiv.org/abs/2412.08465) | **Demographics of M Dwarf Binary Exoplanet Hosts Discovered by TESS**  |
|| R. A. Matson, et al. -- incl., <mark>I. J. M. Crossfield</mark> |
|*Appeared on*| *2024-12-12*|
|*Comments*| *28 pages, 13 figures, 5 tables, accepted for publication in The Astronomical Journal*|
|**Abstract**|            M dwarfs have become increasingly important in the detection of exoplanets and the study of Earth-sized planets and their habitability. However, 20-30% of M dwarfs have companions that can impact the formation and evolution of planetary systems. We use high-resolution imaging and Gaia astrometry to detect stellar companions around M dwarf exoplanet hosts discovered by TESS and determine the projected separation and estimated stellar masses for each system. We find 47 companions around 216 M dwarfs and a multiplicity rate of $19.4\pm2.7$% that is consistent with field M dwarfs. The binary projected separation distribution is shifted to larger separations, confirming the lack of close binaries hosting transiting exoplanets seen in previous studies. We correct the radii of planets with nearby companions and examine the properties of planets in M dwarf multi-star systems. We also note three multi-planet systems that occur in close binaries ($\lesssim 50$ au) where planet formation is expected to be suppressed.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.08527-b31b1b.svg)](https://arxiv.org/abs/2412.08527) | **Characterization of seven transiting systems including four warm Jupiters from SOPHIE and TESS**  |
|| N. Heidari, et al. -- incl., <mark>J. Eberhardt</mark>, <mark>T. Henning</mark> |
|*Appeared on*| *2024-12-12*|
|*Comments*| *46 pages, 21 figures, accepted for publication in A&A*|
|**Abstract**|            We present the study of seven systems, three of which TOI-2295, TOI-2537, and TOI-5110 are newly discovered planetary systems. Through the analysis of TESS photometry, SOPHIE radial velocities, and high-spatial resolution imaging, we found that TOI-2295b, TOI-2537b, and TOI-5110b are transiting warm Jupiters with orbital periods ranging from 30 to 94 d, masses between 0.9 and 2.9 MJ, and radii ranging from 1.0 to 1.5 RJ. Both TOI-2295 and TOI-2537 each harbor at least one additional, outer planet. Their outer planets TOI-2295c and TOI-2537c are characterized by orbital periods of 966.5 +/- 4.3 and 1920^{+230}_{-140} d, respectively, and minimum masses of 5.61^{+0.23}_{-0.24} and 7.2 +/- 0.5 MJ, respectively. We also investigated and characterized the two recently reported warm Jupiters TOI-1836b and TOI-5076b, which we independently detected in SOPHIE RVs. Additionally, we study the planetary candidates TOI-4081.01 and TOI-4168.01. For TOI-4081.01, despite our detection in radial velocities, we cannot rule out perturbation by a blended eclipsing binary and thus exercise caution regarding its planetary nature. On the other hand, we identify TOI-4168.01 as a firm false positive. Finally, we highlight interesting characteristics of these new planetary systems. The transits of TOI-2295b are highly grazing, with an impact parameter of 1.056$^{+0.063}_{-0.043}$. TOI-2537b, in turn, is a temperate Jupiter with an effective temperature of 307+/-15 K and can serve as a valuable low-irradiation control for models of hot Jupiter inflation anomalies. We also detected significant transit timing variations (TTVs) for TOI-2537b, which are likely caused by gravitational interactions with the outer planet TOI-2537c. Finally, TOI-5110b stands out due to its orbital eccentricity of 0.75+/- 0.03, one of the highest planetary eccentricities discovered thus far.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.08057-b31b1b.svg)](https://arxiv.org/abs/2412.08057) | **Enhancement of small-scale induced gravitational waves from the soliton/oscillon domination**  |
|| X.-B. Sui, <mark>J. Liu</mark>, R.-G. Cai |
|*Appeared on*| *2024-12-12*|
|*Comments*| *13 pages, 8 figures*|
|**Abstract**|            We investigate density fluctuations and scalar-induced gravitational waves (GWs) arising from the production of long-lived solitons and oscillons, which can dominate the early Universe and drive reheating prior to the standard radiation-dominated era. Curvature perturbations are generated not only by the Poisson distribution of these solitons/oscillons but are also amplified during the early matter-dominated phase when the solitons/oscillons dominate. Scalar-induced GWs with characteristic energy spectra emerge from these amplified curvature perturbations, particularly at the sudden transition from matter domination to radiation domination. We analyze this scenario and its GW predictions in detail, focusing on the impact of the initial energy fraction, the lifetime, and the average separation of these solitons/oscillons. This provides a wealth of potential observational targets for various GW detectors. Furthermore, for inflation and preheating models that produce oscillons with large separations, we derive new constraints on these models based on the upper bounds on the effective number of relativistic degrees of freedom, as anticipated from future cosmic microwave background and large-scale structure observations.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.08390-b31b1b.svg)](https://arxiv.org/abs/2412.08390) | **Observational studies on S-bearing molecules in massive star forming regions**  |
|| R. Luo, et al. -- incl., <mark>X. Zhang</mark>, <mark>J. Li</mark> |
|*Appeared on*| *2024-12-12*|
|*Comments*| *15 pages, 7 figures*|
|**Abstract**|            Aims. We present observational results of H$_{2}$S 1$_{10}$-1$_{01}$, H$_{2}$$^{34}$S 1$_{10}$-1$_{01}$, H$_{2}$CS 5$_{14}$-4$_{14}$, HCS$^{+}$ 4-3, SiO 4-3, HC$_{3}$N 19-18 and C$^{18}$O 1-0 toward a sample of 51 late-stage massive star-forming regions, to study relationships among H$_{2}$S, H$_{2}$CS, HCS$^{+}$ and SiO in hot cores. Chemical connections of these S-bearing molecules are discussed based on the relations between relative abundances in sources. Results. H$_{2}$S 1$_{10}$-1$_{01}$, H$_{2}$$^{34}$S 1$_{10}$-1$_{01}$, H$_{2}$CS 5$_{14}$-4$_{14}$, HCS$^{+}$ 4-3 and HC$_{3}$N 19-18 were detected in 50 of the 51 sources, while SiO 4-3 was detected in 46 sources. C$^{18}$O 1-0 was detected in all sources. The Pearson correlation coefficients between H$_{2}$CS and HCS$^+$ normalized by H$_{2}$ and H$_{2}$S are 0.94 and 0.87, respectively, and a tight linear relationship is found between them with slope of 1.00 and 1.09, while they are 0.77 and 0.98 between H$_2$S and H$_2$CS, respectively, and 0.76 and 0.97 between H$_2$S and HCS$^+$. The values of full width at half maxima (FWHM) of them in each source are similar to each other, which indicate that they can trace similar regions. Comparing the observed abundance with model results, there is one possible time (2-3$\times$10$^{5}$ yr) for each source in the model. The abundances of these molecules increase with the increment of SiO abundance in these sources, which implies that shock chemistry may be important for them. Conclusions. Close abundance relation of H$_2$S, H$_2$CS and HCS$^+$ molecules and similar line widths in observational results indicate that these three molecules could be chemically linked, with HCS$^+$ and H$_2$CS the most correlated. The comparison of the observational results with chemical models shows that the abundances can be reproduced for almost all the sources at a specific time. The observational results, including abundances in these sources need to be considered in further modeling H$_{2}$S, H$_{2}$CS and HCS$^{+}$ in hot cores with shock chemistry.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.08333-b31b1b.svg)](https://arxiv.org/abs/2412.08333) | **Orbit determination of visual binary systems observed with CST telescope in 2010-2013**  |
|| F. M. Rica, et al. -- incl., <mark>C. Vázquez</mark> |
|*Appeared on*| *2024-12-12*|
|*Comments*| *18 pages, 4 figures containing several plots and 6 tables*|
|**Abstract**|            We present new orbital solutions for 15 binaries, which were astrometrically measured during 2010-2013. We observe the binary systems using the FastCam ``lucky-imaging'' camera, installed at the 1.5-m Carlos Sánchez Telescope (CST) at the Observatorio del Teide, Tenerife (Spain). We present first orbital solutions for four binaries and revise orbits for other 11 systems. We apply two orbital calculation techniques, the ``three-dimensional grid search method'' and the Docobo's analytical method. In addition, we use our tool ``Binary Deblending'', which is based on deblending the entire observed multiband photometry into fundamental and photometric parameters for each stellar component based on PARSEC isochrones. This method allows us to obtain the total mass for all systems. Our findings include the identification of a binary system consisting of two M-type dwarfs (WOR 19), a binary of evolved components (twin F6IV-V stars) in BU 1292, accompanied by a newly discovered wide (10.5 arcsec) and faint companion with G=17.05 mag. Additionally, we explore the X-ray emission system STF 147 and a very young quadruple system, WDS 04573+5345. This comprehensive analysis significantly contributes to our understanding on the formation and evolution of these stellar systems.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2412.08465/./Final_Separation_and_Delta_Mag_of_Gaia_Companions.png', 'tmp_2412.08465/./Final_Sep_and_Per_Histograms.png', 'tmp_2412.08465/./Final_HR_Diagram.png']
copying  tmp_2412.08465/./Final_Separation_and_Delta_Mag_of_Gaia_Companions.png to _build/html/
copying  tmp_2412.08465/./Final_Sep_and_Per_Histograms.png to _build/html/
copying  tmp_2412.08465/./Final_HR_Diagram.png to _build/html/
exported in  _build/html/2412.08465.md
    + _build/html/tmp_2412.08465/./Final_Separation_and_Delta_Mag_of_Gaia_Companions.png
    + _build/html/tmp_2412.08465/./Final_Sep_and_Per_Histograms.png
    + _build/html/tmp_2412.08465/./Final_HR_Diagram.png
found figures ['tmp_2412.08527/./tpf_toi1836.png', 'tmp_2412.08527/./tpf_toi2295.png', 'tmp_2412.08527/./tpf_toi2537.png', 'tmp_2412.08527/./tpf_toi4081.png', 'tmp_2412.08527/./4168/tpf_toi4168.png', 'tmp_2412.08527/./tpf_toi5076.png', 'tmp_2412.08527/./tpf_toi5110.png', 'tmp_2412.08527/./TPF_Gaia_TIC207468071_S23.png', 'tmp_2412

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{Ç}[1]$</div>



<div id="title">

# Demographics of M Dwarf Binary Exoplanet Hosts Discovered by TESS

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2412.08465-b31b1b.svg)](https://arxiv.org/abs/2412.08465)<mark>Appeared on: 2024-12-12</mark> -  _28 pages, 13 figures, 5 tables, accepted for publication in The Astronomical Journal_

</div>
<div id="authors">

R. A. Matson, et al. -- incl., <mark>I. J. M. Crossfield</mark>

</div>
<div id="abstract">

**Abstract:** M dwarfs have become increasingly important in the detection of exoplanets and the study of Earth-sized planets and their habitability. However, 20 $-$ 30 \% of M dwarfs have companions that can impact the formation and evolution of planetary systems. We use high-resolution imaging and Gaia astrometry to detect stellar companions around M dwarf exoplanet hosts discovered by TESS and determine the projected separation and estimated stellar masses for each system. We find 47 companions around 216 M dwarfs and a multiplicity rate of $19.4\pm2.7$ \% that is consistent with field M dwarfs. The binary projected separation distribution is shifted to larger separations, confirming the lack of close binaries hosting transiting exoplanets seen in previous studies.We correct the radii of planets with nearby companions and examine the properties of planets in M dwarf multi-star systems. We also note three multi-planet systems that occur in close binaries ( $\lesssim 50$ au) where planet formation is expected to be suppressed.

</div>

<div id="div_fig1">

<img src="tmp_2412.08465/./Final_Separation_and_Delta_Mag_of_Gaia_Companions.png" alt="Fig5" width="100%"/>

**Figure 5. -** _Left_: Magnitude difference as a function of angular separation for companions detected via common proper motion and parallax using Gaia DR3. Companions also detected with high-resolution imaging are shown in light blue. _Right_: Companions detected via Gaia DR3 and the CPM companions of TOI-2221 are plotted in blue, with companions detected using high-resolution imaging and not also detected using Gaia shown in orange. Open circles are companions unresolved by Gaia, or with no proper motion and/or parallax data, which are assumed to be bound (see text for details). Blue $\times$'s show companions present in Gaia with parallax and proper motion data indicating they are not physically bound. Each companion is only plotted once, with overlapping companions a result of the magnitude differences measured in different filters. Comparing the separation ranges of the high resolution and CPM detections demonstrates that while there is overlap, they provide complementary samples. (*fig:Gaiacomps*)

</div>
<div id="div_fig2">

<img src="tmp_2412.08465/./Final_Sep_and_Per_Histograms.png" alt="Fig10" width="100%"/>

**Figure 10. -** _Left:_ The projected separation distribution in au for M dwarf TOI's with companions. The subset of companions detected with high-resolution (HR) is highlighted in light blue. We converted the observed separations of each system from arcseconds to  au using parallaxes from Gaia, then fit both the total sample (dot dash black line) and the high-resolution subset with Gaussian curves with $\sigma =$ 1.06 and $\sigma =$ 0.65 $\log$(AU), respectively. For the high-resolution sample, the peak is $\mu =$ 66 au,  whereas for the total sample, the peak is $\mu =$ 596 au. The separation distributions of non-planet hosting, planet hosting, and TOI M stars from 2024AJ....167..174C are shown for comparison. The region shaded in gray highlights the inner angular resolution limits of the various high-resolution imaging techniques at 75pc, the mean distance for M dwarfs in our sample. These limits demonstrate that we are sensitive to stellar companions with separations of a few au where the M-dwarf distribution of 2024AJ....167..174C peaks.
    _Right:_ A normalized cumulative distribution function for the projected companion separation in au. The function highlights the deficit of companions within 45 au (indicated by the vertical dashed line). Half of the M dwarf TOI companions are within 500  au (denoted by the vertical dotted line), reflecting the high concentration of companions between $45-500$ au as seen in the histogram on the left. The change in slope at $\sim50$ au is consistent with the semi-major axis cutoffs for close binaries with exoplanets determined by [Kraus, et. al (2016)]() and [Ziegler, Tokovinin and Latiolais (2021)]().
     (*fig:logsep*)

</div>
<div id="div_fig3">

<img src="tmp_2412.08465/./Final_HR_Diagram.png" alt="Fig1" width="100%"/>

**Figure 1. -** H-R Diagram of TESS TOI candidates. Grey symbols are randomly chosen F, G, K stars taken from the TOI list and plotted here to give a perspective for their locations in an H-R Diagram. Red dots are M star TOIs which have already been deemed to be false positives, while blue dots show the location of the remaining M star TOI candidates. The blue dots above the main sequence are deemed to be giants and are removed from any subsequent analysis. (*fig:hrdiagram*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2412.08465"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\Tau}{\mathrm{T}}$
$\newcommand$
$\newcommand{\exofasttwo}{{\tt EXOFASTv2}}$
$\newcommand{\bjdtdb}{\ensuremath{\rm{BJD_{TDB}}}}$
$\newcommand{\tjdtdb}{\ensuremath{\rm{TJD_{TDB}}}}$
$\newcommand{\msun}{\ensuremath{ M_\odot}}$
$\newcommand{\rsun}{\ensuremath{ R_\odot}}$
$\newcommand{\lsun}{\ensuremath{ L_\odot}}$
$\newcommand{\thefigure}{\arabic{figure}.}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.2}$</div>



<div id="title">

# Characterization of seven transiting systems including four warm Jupiters from SOPHIE and TESS

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2412.08527-b31b1b.svg)](https://arxiv.org/abs/2412.08527)<mark>Appeared on: 2024-12-12</mark> -  _46 pages, 21 figures, accepted for publication in A&A_

</div>
<div id="authors">

N. Heidari, et al. -- incl., <mark>J. Eberhardt</mark>, <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** While several thousand exoplanets are now confirmed, the number of known transiting warm Jupiters ( $10  \text{d} < \text{period} < 200   \text{d}$ ) remains relatively small. These planets are generally believed to have formed outside the snowline and migrated to their current orbits. Because they are sufficiently distant from their host stars, they mitigate proximity effects and so offer valuable insights into planet formation and evolution. Here, we present the study of seven systems, three of which—TOI-2295, TOI-2537, and TOI-5110—are newly discovered planetary systems. Through the analysis of TESS photometry, SOPHIE radial velocities, and high-spatial resolution imaging, we found that TOI-2295b, TOI-2537b, and TOI-5110b are transiting warm Jupiters with orbital periods ranging from 30 to 94 d, masses between 0.9 and 2.9 $M_{\rm{J}}$ , and radii ranging from 1.0 to 1.5 $R_{\rm{J}}$ . Both TOI-2295 and TOI-2537 each harbor at least one additional, outer planet. Their outer planets —TOI-2295c and TOI-2537c— are characterized by orbital periods of 966.5 $^{+4.3}_{-4.2}$ and  1920 $^{+230}_{-140}$ d, respectively, and minimum masses of 5.61 $^{+0.23}_{-0.24}$ and 7.23 $^{+0.52}_{-0.45}$ $M_{\rm{J}}$ , respectively. We also investigated and characterized the two recently reported warm Jupiters TOI-1836b and TOI-5076b, which we independently detected in SOPHIE RVs. Our new data allow for further discussion of their nature and refinement of their parameters. Additionally, we study the planetary candidates TOI-4081.01 and TOI-4168.01. For TOI-4081.01, despite our detection in radial velocities, we cannot rule out perturbation by a blended eclipsing binary and thus exercise caution regarding its planetary nature. On the other hand, we identify TOI-4168.01 as a firm false positive; its radial velocity curve exhibits a large amplitude in an anti-phase relation with the transit ephemeris observed by TESS, indicating that the detected event is the eclipse of a secondary star rather than a planetary transit. Finally, we highlight interesting characteristics of these new planetary systems. The transits of TOI-2295b are highly grazing, with an impact parameter of 1.056 $^{+0.063}_{-0.043}$ . This leaves its radius uncertain but potentially makes it an interesting probe of gravitational dynamics in its two-planet system, as transit shapes for grazing planets are highly sensitive to even small variations in inclination. TOI-2537b, in turn, is a temperate Jupiter with an effective temperature of 307 $\pm$ 15 K and can serve as a valuable low-irradiation control for models of hot Jupiter inflation anomalies. We also detected significant transit timing variations (TTVs) for TOI-2537b, which are likely caused by gravitational interactions with the outer planet TOI-2537c. Further transit observations are needed to refine the analysis of these TTVs and enhance our understanding of the system’s dynamics. Finally, TOI-5110b stands out due to its orbital eccentricity of 0.745 $^{+0.030}_{-0.027}$ , one of the highest planetary eccentricities discovered thus far. We find no conclusive evidence for an external companion, but an unseen planet with a semi-amplitude smaller than 10 m/s could nonetheless still be exciting its eccentricity.

</div>

<div id="div_fig1">

<img src="tmp_2412.08527/./tpf_toi1836.png" alt="Fig19.1" width="14%"/><img src="tmp_2412.08527/./tpf_toi2295.png" alt="Fig19.2" width="14%"/><img src="tmp_2412.08527/./tpf_toi2537.png" alt="Fig19.3" width="14%"/><img src="tmp_2412.08527/./tpf_toi4081.png" alt="Fig19.4" width="14%"/><img src="tmp_2412.08527/./4168/tpf_toi4168.png" alt="Fig19.5" width="14%"/><img src="tmp_2412.08527/./tpf_toi5076.png" alt="Fig19.6" width="14%"/><img src="tmp_2412.08527/./tpf_toi5110.png" alt="Fig19.7" width="14%"/>

**Figure 19. -** Target Pixel Files (TPFs) images from the first observed TESS sector for the seven targets in our analysis, created by \texttt{tpfplotter} ([Aller, et. al 2020]()) . The images depict electron counts, with red-contoured pixels marking the SPOC aperture mask. Additionally, red circles highlight the primary target (numbered as 1) and nearby sources (rest of the numbers) at their Gaia DR3 positions. The area of these circles reflects the relative magnitudes of the sources in comparison to the target star. Arrows represent the proper motion of each star. (*tpfplotter*)

</div>
<div id="div_fig2">

<img src="tmp_2412.08527/./TPF_Gaia_TIC207468071_S23.png" alt="Fig23.1" width="9%"/><img src="tmp_2412.08527/./TPF_Gaia_TIC207468071_S24.png" alt="Fig23.2" width="9%"/><img src="tmp_2412.08527/./TPF_Gaia_TIC207468071_S25.png" alt="Fig23.3" width="9%"/><img src="tmp_2412.08527/./TPF_Gaia_TIC207468071_S49.png" alt="Fig23.4" width="9%"/><img src="tmp_2412.08527/./TPF_Gaia_TIC207468071_S50.png" alt="Fig23.5" width="9%"/><img src="tmp_2412.08527/./TPF_Gaia_TIC207468071_S51.png" alt="Fig23.6" width="9%"/><img src="tmp_2412.08527/./TPF_Gaia_TIC207468071_S52.png" alt="Fig23.7" width="9%"/><img src="tmp_2412.08527/./TPF_Gaia_TIC207468071_S56.png" alt="Fig23.8" width="9%"/><img src="tmp_2412.08527/./TPF_Gaia_TIC207468071_S76.png" alt="Fig23.9" width="9%"/><img src="tmp_2412.08527/./TPF_Gaia_TIC207468071_S77.png" alt="Fig23.10" width="9%"/><img src="tmp_2412.08527/./TPF_Gaia_TIC207468071_S78.png" alt="Fig23.11" width="9%"/>

**Figure 23. -** The TPFs for TOI-1836, created by \texttt{tpfplotter} ([Aller, et. al 2020]()) . The images display electron counts, with the SPOC aperture mask outlined in red. Red circles are used to indicate the primary target (marked as 1) and nearby sources (rest of the numbers) based on their positions in Gaia DR3. The size of these circles corresponds to the relative magnitudes of the sources compared to the target star. Additionally, arrows illustrate the proper motion of each star. (*tpfplotter_1836*)

</div>
<div id="div_fig3">

<img src="tmp_2412.08527/./high_image_all.png" alt="Fig20" width="100%"/>

**Figure 20. -** Contrast curves for the speckle interferometry observations of TOI-1836, TOI-2295, TOI-2537, TOI-4081, TOI-4168, TOI-5076, and TOI-5110 are shown with black, blue, and red solid lines. The name of each star is indicated on the corresponding plot. TOI-1836, TOI-4081, TOI-4168, TOI-5076, and TOI-5110 were observed by SPP in 625 or 880 nm band. Meanwhile, TOI-2295 was observed using Gemini 'Alopeke, which provides simultaneous speckle imaging in two bands: 562 nm (blue line) and 832 nm (red line). Additionally, TOI-2537 was observed by SOAR in 880 nm band. Each plot includes the final reconstructed image inset in the upper right corner. (*spp*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2412.08527"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

280  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

15  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

6  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
